In [1]:
##Estimación de pobreza de adecuación energética de acuerdo con Evalúa CDMX
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr,showtext,gt)


#Fuente para los gráficos
font_add_google("Poppins", "pop")
showtext_auto()

Cargando paquete requerido: pacman



In [2]:
# Definir URL base y parte intermedia
url_basica <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte <- "microdatos/enigh"

# Definir los años y los tipos de archivos a descargar
years <- c("2018", "2020", "2022","2024")
tipos <- c("ns_viviendas", "ns_poblacion")  # Tipos de archivos

# Crear la carpeta de destino si no existe
if (!dir.exists("data")) {
  dir.create("data")
}

# Descargar, descomprimir y renombrar los archivos
for (year in years) {
  for (tipo in tipos) {
    # Construir URL y nombres de archivo
    url <- paste0(url_basica, year, "/", segunda_parte, year, "_", tipo, "_csv.zip")
    zip_destfile <- paste0("data/", year, "_", tipo, ".zip")

    # Descargar solo si el archivo no existe
    if (!file.exists(zip_destfile)) {
      download.file(url, destfile = zip_destfile, mode = "wb")
    } else {
      message("Archivo ya descargado: ", zip_destfile)
    }

    # Extraer archivos en una subcarpeta específica
    extract_dir <- paste0("data/", year, "_", tipo)
    if (!dir.exists(extract_dir)) {
      dir.create(extract_dir)
    }

    unzip(zip_destfile, exdir = extract_dir)

    # Buscar el archivo CSV dentro de la carpeta extraída
    extracted_files <- list.files(extract_dir, pattern = "\\.csv$", full.names = TRUE)

    if (length(extracted_files) == 1) {
      # Determinar el nuevo nombre con formato adecuado
      new_name <- paste0("data/", tipo, "_", year, ".csv")

      # Mover el archivo al directorio principal y renombrarlo
      file.rename(extracted_files, new_name)

      message("Archivo renombrado y movido: ", new_name)
    } else {
      message("Error: No se encontró un archivo CSV único en ", extract_dir)
    }
  }
}


Archivo ya descargado: data/2018_ns_viviendas.zip

Archivo renombrado y movido: data/ns_viviendas_2018.csv

Archivo ya descargado: data/2018_ns_poblacion.zip

Archivo renombrado y movido: data/ns_poblacion_2018.csv

Archivo ya descargado: data/2020_ns_viviendas.zip

Archivo renombrado y movido: data/ns_viviendas_2020.csv

Archivo ya descargado: data/2020_ns_poblacion.zip

Archivo renombrado y movido: data/ns_poblacion_2020.csv

Archivo ya descargado: data/2022_ns_viviendas.zip

Archivo renombrado y movido: data/ns_viviendas_2022.csv

Archivo ya descargado: data/2022_ns_poblacion.zip

Archivo renombrado y movido: data/ns_poblacion_2022.csv

Archivo ya descargado: data/2024_ns_viviendas.zip

Archivo renombrado y movido: data/ns_viviendas_2024.csv

Archivo ya descargado: data/2024_ns_poblacion.zip

Archivo renombrado y movido: data/ns_poblacion_2024.csv



In [3]:
estima_pobreza_energetica <- function(year, variable) {

  # Procesar vivienda 
  vivienda <- read_csv(paste0("data/ns_viviendas_", year, ".csv"), show_col_types = FALSE) %>%
    {
      if (year <= 2022) {
        mutate(.,
          disp_elect = as.numeric(disp_elect),
          focos_inca = as.numeric(focos_inca),
          focos_ahor = as.numeric(focos_ahor),
          num_cuarto = as.numeric(num_cuarto),
          bano_comp  = as.numeric(bano_comp),
          bano_excus = as.numeric(bano_excus),
          bano_regad = as.numeric(bano_regad),
          combustible = as.numeric(combustible)
        )
      } else if (year >= 2024) {
        mutate(.,
          disp_elect = as.numeric(disp_elect),
          focos      = as.numeric(focos),
          focos_ahor = as.numeric(focos_ahor),
          num_cuarto = as.numeric(num_cuarto),
          bano_comp  = as.numeric(bano_comp),
          bano_excus = as.numeric(bano_excus),
          bano_regad = as.numeric(bano_regad),
          combus     = as.numeric(combus)
        )
      }
    } %>%
    mutate(
      AElj = case_when(
        disp_elect == 5 ~ 0,              # no tiene luz eléctrica
        disp_elect %in% c(3, 4) ~ 0.5,    # panel solar u otra fuente
        disp_elect %in% c(1, 2) ~ 1,      # servicio público / planta particular
        TRUE ~ NA_real_
      )
    ) %>%

    # --- Logro de focos ---
    mutate(
      CTj = num_cuarto,

      Focos = if (year <= 2022) {
        replace_na(focos_inca, 0) + replace_na(focos_ahor, 0)
      } else if (year >= 2024) {
        replace_na(focos, 0)
      },

      # baños: (.=0) y conteo
      bano_comp  = replace_na(bano_comp,  0),
      bano_excus = replace_na(bano_excus, 0),
      bano_regad = replace_na(bano_regad, 0),
      num_bano   = bano_comp + bano_excus + bano_regad,

      # cuartos totales incluyendo baños
      CTBj = CTj + num_bano,

      # norma (focos por cuarto incluyendo baños)
      Fj_N = 1.25,

      # indicador logro de focos
      LFj  = if_else(CTBj > 0, (Focos / CTBj) / Fj_N, NA_real_),

      # reescalado: máximo 2
      LFj_P = if_else(LFj > 1, pmin(1 + (LFj - 1) / 3, 2), LFj),

      # indicador integral de logro de electricidad y su privación
      LIEI = AElj * LFj_P,
      PIEI = 1 - LIEI
    ) %>%

    # --- Fuente de energía para cocinar (2024+: 'combus' con 7 = No cocinan) ---
    mutate(
      comb_num = if (year >= 2024) combus else combustible,
      CK = case_when(
        comb_num %in% c(3, 4, 5) ~ 3,        # gas tanque / natural / electricidad (norma)
        comb_num %in% c(1, 2, 6) ~ 1,        # leña, carbón, otro
        comb_num == 7            ~ 3,        # NO cocinan
        TRUE ~ NA_real_
      ),
      ACK = CK / 3,
      PCK = 1 - ACK,

      # adecuación energética y carencia
      AENj = ACK * 0.30 + LIEI * 0.70,
      CENj = 1 - AENj,
      year = year
    ) %>%

    # Selección de variables
    {
      if (year < 2022) select(., folioviv, AElj, LFj_P, LIEI, PIEI, ACK, PCK, AENj,CTBj ,CENj,Focos,comb_num, factor, upm, est_dis) else
        select(., folioviv, AElj, LFj_P, LIEI, PIEI, ACK, PCK, AENj, CENj,Focos,comb_num,CTBj)
    }

  # Población
  pob <- read_csv(paste0("data/ns_poblacion_", year, ".csv"), show_col_types = FALSE) %>%
    janitor::clean_names() %>%
    # Filtrar parentescos que no pertenecen al núcleo familiar
    filter(!(parentesco >= 400 & parentesco < 500) & !(parentesco >= 700 & parentesco < 800)) %>%
    # Crear clave de entidad
    mutate(
      cve_ent = case_when(
        nchar(folioviv) == 9 ~ paste0("0", substr(folioviv, 1, 1)),
        nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
      )
    ) %>%
    # Pegar vivienda
    left_join(vivienda, by = "folioviv") %>%
    mutate(
      # Estratos de carencia energética
      E_CENJ = case_when(
        CENj <= -0.5 ~ 6,
        CENj > -0.5 & CENj <= -0.1 ~ 5,
        CENj > -0.1 & CENj <= 0 ~ 4,
        CENj > 0 & CENj <= (1/3) ~ 3,
        CENj > (1/3) & CENj <= 0.5 ~ 2,
        CENj > 0.5 & CENj <= 1 ~ 1,
        TRUE ~ NA_real_
      ),
      # Pobreza energética
      pobreza_cenj = case_when(
        E_CENJ >= 4 ~ 0,
        E_CENJ <= 3 ~ 1,
        TRUE ~ NA_real_
      )
    )

  # Diseño muestral
  diseno <- pob %>%
    as_survey_design(
      ids = upm,
      strata = est_dis,
      weights = factor,
      nest = TRUE
    )

  # Estimaciones
  resultado <- if (variable == "pobreza_energetica") {
    diseno %>%
      filter(cve_ent == "09") %>%
      group_by(pobreza_cenj) %>%
      summarise(
        pob = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
      ) %>%
      mutate(porcentaje = porcentaje * 100, year = year) %>%
      filter(pobreza_cenj == 1)
  
  } else {
    stop("Variable no reconocida")
  }

  return(resultado)
}

In [4]:
tabla_pobreza <- map_dfr(years, estima_pobreza_energetica, variable = "pobreza_energetica")
tabla_pobreza

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or mo

pobreza_cenj,pob,pob_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,4412792,0.03827475,47.98463,0.03202021,2018
1,4576102,0.03400964,49.52146,0.02907445,2020
1,4335357,0.03611505,46.59415,0.03099795,2022
1,3987891,0.04050515,42.67149,0.03468308,2024


In [6]:
tabla_pobreza %>%
  select(-pobreza_cenj) %>%
  rename(
    Año           = year,
    Personas      = pob,
    Personas_cv   = pob_cv,
    Porcentaje    = porcentaje,
    Porcentaje_cv = porcentaje_cv
  ) %>%
  relocate(Año, .before = Personas) %>%
  gt() %>%
  # textos de la segunda fila de encabezados
  cols_label(
    Personas      = "Valor",
    Personas_cv   = "Coeficiente de variación",
    Porcentaje    = "Valor",
    Porcentaje_cv = "Coeficiente de variación"
  ) %>%
  tab_spanner("Personas",   id = "sp_personas",   columns = c(Personas, Personas_cv)) %>%
  tab_spanner("Porcentaje", id = "sp_porcentaje", columns = c(Porcentaje, Porcentaje_cv)) %>%

  # --- Poppins + centrado ---
  opt_table_font(font = list(google_font("Poppins"), default_fonts())) %>%   # fuente
  tab_options(
    heading.align = "center",
    table.align   = "center",
    table.font.size = px(14),
    table.background.color = "white",
    table.border.top.color = "white",
    table.border.bottom.color = "white",
    data_row.padding = px(3),
    column_labels.font.weight = "bold",
    column_labels.border.top.color = "black",
    column_labels.border.bottom.color = "black",
    column_labels.border.bottom.width = px(2),
    table_body.hlines.color = "lightgrey",
    table_body.hlines.width = px(0.5),
    table_body.border.bottom.color = "black",
    table_body.border.bottom.width = px(2),
    source_notes.font.size = px(10),
    source_notes.padding = px(1)
  ) %>%
  # centrar celdas, labels y spanners
  cols_align(align = "center", columns = everything()) %>%
  tab_style(
    style = cell_text(align = "center"),
    locations = list(
      cells_column_labels(columns = everything()),
      cells_column_spanners(spanners = everything())
    )
  ) %>%

  fmt_number(columns = c(Personas, Personas_cv, Porcentaje, Porcentaje_cv), decimals = 2) %>%
  tab_header(
    title = md("**Pobreza de adecuación energética en CDMX**"),
    subtitle = md("**Personas en pobreza de adecuación energética (%)**")
  ) %>%
  tab_source_note(md("Nota: Resultados preliminares de acuerdo con la metodología de Evalúa CDMX")) %>%
  tab_source_note(md("Fuente: @claudiodanielpc con información de INEGI. Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH)")) %>%
  gtsave("pobreza_energetica.png")